In [1]:
from mapper import write_time_based_train_test_split
from mapper import map_to_PGPR

In [2]:
dataset_name = 'lfm'

In [3]:
write_time_based_train_test_split(dataset_name, "pgpr", 0.8)

In [4]:
map_to_PGPR('lfm')

In [1]:
%cd models/PGPR


/home/jfallmann/userdata/Bachelor Thesis/bachelor-thesis/src/models/PGPR


In [6]:
%run preprocess.py --dataset lfm

Load lfm dataset from file...
Load user of size 9455
Load product of size 173788
Load artist of size 14987
Load album of size 45978
Load genre of size 749
Load micro_genre of size 1921
Load in_album of size 111577
Load has_micro_genre of size 667023
Load created_by of size 111577
Load has_genre of size 111577
0 0
Load review of size 3941518 with positive reviews= 0  and negative reviews= 3941518
Create lfm knowledge graph from dataset...
Load entities...
Total 246884 nodes.
Load reviews...
Total 7883036 review edges.
Load knowledge created_by...
Total 223154 created_by edges.
Load knowledge has_micro_genre...
Total 1334046 has_micro_genre edges.
Load knowledge has_genre...
Total 223154 has_genre edges.
Load knowledge in_album...
Total 223154 in_album edges.
Remove duplicates...
Compute node degrees...
Generate lfm train/test labels.


In [2]:
%run train_transe_model --dataset lfm

../../data/lfm/preprocessed/pgpr/tmp
[INFO]  Namespace(dataset='lfm', name='train_transe_model', seed=123, gpu='0', epochs=30, batch_size=64, lr=0.5, weight_decay=0, l2_lambda=0, max_grad_norm=5.0, embed_size=100, num_neg_samples=5, steps_per_checkpoint=200, device=device(type='cuda', index=0), log_dir='../../data/lfm/preprocessed/pgpr/tmp/train_transe_model')
Load embeddings ../../data/lfm/preprocessed/pgpr/tmp/train_transe_model/transe_model_sd_epoch_30.ckpt


In [2]:
%run train_agent --dataset lfm

[INFO]  Namespace(dataset='lfm', name='train_agent', seed=123, gpu='0', epochs=50, batch_size=32, lr=0.0001, max_acts=250, max_path_len=3, gamma=0.99, ent_weight=0.001, act_dropout=0, state_history=1, hidden=[512, 256], device=device(type='cuda', index=0), log_dir='../../data/lfm/preprocessed/pgpr/tmp/train_agent')
Load embedding: ../../data/lfm/preprocessed/pgpr/tmp/transe_embed.pkl
[INFO]  Parameters:['l1.weight', 'l1.bias', 'l2.weight', 'l2.bias', 'actor.weight', 'actor.bias', 'critic.weight', 'critic.bias']


/home/jfallmann/userdata/Bachelor Thesis/bachelor-thesis/src/models/PGPR/train_agent.py:50: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/aten/src/ATen/native/cuda/Indexing.cu:1440.)
  actor_logits[1 - act_mask] = -999999.0
/home/jfallmann/miniconda3/envs/bsc/lib/python3.11/site-packages/torch/autograd/__init__.py:200: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/aten/src/ATen/native/cuda/Indexing.cu:1440.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


[INFO]  epoch/step=1/100 | loss=0.37647 | ploss=0.26276 | vloss=0.12815 | entropy=-14.43317 | reward=0.16084
[INFO]  epoch/step=1/200 | loss=0.18247 | ploss=0.07980 | vloss=0.11705 | entropy=-14.36958 | reward=0.15693
[INFO]  epoch/step=2/300 | loss=0.26236 | ploss=0.15892 | vloss=0.11780 | entropy=-14.35787 | reward=0.16377
[INFO]  epoch/step=2/400 | loss=0.14128 | ploss=0.04370 | vloss=0.11196 | entropy=-14.37236 | reward=0.16016
[INFO]  epoch/step=2/500 | loss=0.14769 | ploss=0.05588 | vloss=0.10618 | entropy=-14.36755 | reward=0.15888
[INFO]  epoch/step=3/600 | loss=0.16835 | ploss=0.08030 | vloss=0.10241 | entropy=-14.36520 | reward=0.15782
[INFO]  epoch/step=3/700 | loss=0.19628 | ploss=0.10005 | vloss=0.11065 | entropy=-14.42180 | reward=0.16272
[INFO]  epoch/step=3/800 | loss=0.15397 | ploss=0.05865 | vloss=0.10971 | entropy=-14.38560 | reward=0.16330
[INFO]  epoch/step=4/900 | loss=0.16999 | ploss=0.07735 | vloss=0.10706 | entropy=-14.41882 | reward=0.15938
[INFO]  epoch/step=

In [5]:
%run test_agent --dataset lfm

Predicting paths...
Load embedding: ../../data/lfm/preprocessed/pgpr/tmp/transe_embed.pkl



100%|█████████▉| 9440/9455 [6:17:33<00:28,  1.92s/it]
9456it [6:18:42,  2.40s/it]                          


Load embedding: ../../data/lfm/preprocessed/pgpr/tmp/transe_embed.pkl
Normalizing items scores...
Saving pred_paths...
Average test set size:  104.71782125859333
Overall for noOfUser=9455, ndcg=0.0906


Overall for noOfUser=9455, hr=0.1687


Overall for noOfUser=9455, precision=0.0222


Overall for noOfUser=9455, recall=0.0025




In [3]:
import numpy as np
import pandas as pd
from random import seed, randint, choice
from collections import defaultdict
import pickle

In [2]:
with open("results/lfm/pgpr/pred_paths.pkl", 'rb') as pred_paths_file:
    pred_paths_pgpr = pickle.load(pred_paths_file)
pred_paths_file.close()

In [4]:
header = ["uid", "pid", "path_score", "path_prob", "path"]
pred_paths_map_pgpr = defaultdict(dict)
for record in pred_paths_pgpr:
    uid, pid, path_score, path_prob, path = record
    if pid not in pred_paths_map_pgpr[uid]:
        pred_paths_map_pgpr[uid][pid] = []
    pred_paths_map_pgpr[uid][pid].append((float(path_score), float(path_prob), path))

In [9]:
n_users = len(pred_paths_map_pgpr.keys())
random_user = randint(0, n_users)
random_product = choice(list(pred_paths_map_pgpr[random_user].keys()))
print(pred_paths_map_pgpr[random_user][random_product])
print(len(pred_paths_map_pgpr[random_user]))

[(0.557691845909847, 2.5638131774030626e-05, 'self_loop user 9163 listened_to product 40838 has_micro_genre micro_genre 961 has_micro_genre product 167643'), (0.557691845909847, 9.663195669418201e-06, 'self_loop user 9163 listened_to product 95256 has_micro_genre micro_genre 961 has_micro_genre product 167643')]
445
